### Step 3: Prune the fine-tuned teacher model to create a student
In this step, we will explore two methods to prune the fine-tuned teacher model. Refer to the ``NOTE`` in the **_step-by-step instructions_** section of [introduction.ipynb](./introduction.ipynb) to decide which pruning techniques you would like to explore.

In the first method, depth-pruning, we trim the layers of the model.

#### Step 3.a.: Using depth-pruning 
To depth-prune, we will trim the layers 16-31 (leaving 1-15 and 32) in the finetined teacher model. For depth-pruning, we would be using the [megatron_gpt_prune](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/language_modeling/megatron_gpt_prune.py) script. 

Per the [blog](https://developer.nvidia.com/blog/how-to-prune-and-distill-llama-3-1-8b-to-an-nvidia-llama-3-1-minitron-4b-model/) and [tech report](https://arxiv.org/pdf/2408.11796), removing contiguous layers from the second last block (layers 16 to 31 continuously) yields the best overall results. 

> `NOTE:` In the block of code below, pass the paths to your fine-tuned teacher .nemo model.

In [ ]:
!torchrun --nproc_per_node=8 \
     /opt/NeMo/examples/nlp/language_modeling/megatron_gpt_prune.py \
     model.restore_from_path="./distill_trainings/megatron_llama_ft/checkpoints/megatron_llama_ft.nemo" \
     model.tensor_model_parallel_size=8 \
     model.pipeline_model_parallel_size=1 \
     +model.dist_ckpt_load_strictness=log_all \
     trainer.num_nodes=1 \
     trainer.precision=bf16 \
     trainer.devices=8 \
     "prune.drop_layers=[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]" \
     export.save_path="/workspace/4b_depth_pruned_model.nemo"

Running this script will save the depth-pruned model `4b_depth_pruned_model.nemo` to your workspace.